In [13]:
-- 이진트리
data Tree2 a
  = Null2
  | Node2 a (Tree2 a) (Tree2 a)
  deriving Show

-- 임의 개수 가지치는 나무
data Tree a = Node a [Tree a]  deriving Show

In [23]:
-- Tree2의 중위순서(inorder)로 원소를 나열하는 리스트를 계산하자
inOrder2 :: Tree2 a -> [a]
inOrder2 Null2           = []
inOrder2 (Node2 x t1 t2) = inOrder2 t1 ++ [x] ++ inOrder2 t2

In [24]:
-- Tree2의 전위순서(preorder)로 원소를 나열하는 리스트를 계산하자
preOrder2 :: Tree2 a -> [a]
preOrder2 Null2           = []
preOrder2 (Node2 x t1 t2) = x : preOrder2 t1 ++ preOrder2 t2

In [25]:
tree2 = Node2 1
          (Node2 2 (Node2 4 Null2 Null2)
                   (Node2 5 Null2 Null2))
          (Node2 3 (Node2 6 Null2 Null2)
                   (Node2 7 Null2 Null2))
inOrder2 tree2
preOrder2 tree2

[4,2,5,1,6,3,7]

[1,2,4,5,3,6,7]

In [30]:
-- Null2가 아닌 경우 Tree2를 Tree로 변환
fromTree2toTree Null2 = error "Null2 cannot be Tree"
fromTree2toTree (Node2 x t1 t2) = Node x ts
  where
  ts = case (t1, t2) of
         (Null2, Null2) -> []
         (Null2, _    ) -> [t2']
         (_,     Null2) -> [t1']
         (_,     _    ) -> [t1',t2']    
  t1' = fromTree2toTree t1
  t2' = fromTree2toTree t2

In [32]:
tree = fromTree2toTree tree2

tree

Node 1 [Node 2 [Node 4 [],Node 5 []],Node 3 [Node 6 [],Node 7 []]]

In [33]:
preOrder :: Tree a -> [a]
preOrder (Node x ts) = x : [preOrder t | t<-ts]

<interactive>:2:29: error:
    • Couldn't match expected type ‘a’ with actual type ‘[a]’
      ‘a’ is a rigid type variable bound by
        the type signature for:
          preOrder :: forall a. Tree a -> [a]
        at <interactive>:1:1-25
    • In the expression: preOrder t
      In the second argument of ‘(:)’, namely ‘[preOrder t | t <- ts]’
      In the expression: x : [preOrder t | t <- ts]
    • Relevant bindings include
        t :: Tree a (bound at <interactive>:2:42)
        ts :: [Tree a] (bound at <interactive>:2:18)
        x :: a (bound at <interactive>:2:16)
        preOrder :: Tree a -> [a] (bound at <interactive>:2:1)